In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/my-jd-comment-labels/jd_comment_data.xlsx


In [4]:
import torch
import torch.nn as nn
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split

import pandas as pd
from tqdm import tqdm 

2025-06-06 07:50:25.306781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749196225.481207      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749196225.531817      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
writer = SummaryWriter()
device

'cuda'

In [6]:
df = pd.read_excel('/kaggle/input/my-jd-comment-labels/jd_comment_data.xlsx')

df.head()

,爬取时间(__time),爬取链接(__url),商品ID(product_id),评价时间(publish_time),评分（总分5分）(score),评价内容(content),评价者(author_name),评价者会员等级(author_level),商品sku(product_sku),评价标签(tags)
0,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550631798,5,此用户未填写评价内容,j***e,注册会员,单机版 小D黑色,[]
1,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633151,5,此用户未填写评价内容,c***n,钻石会员,单机版 小D黑色,[]
2,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633330,3,此用户未填写评价内容,j***1,银牌会员,单机版 小D黑色,[]
3,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633401,5,此用户未填写评价内容,苗***4,钻石会员,单机版 小D黑色,[]
4,2019-03-08 00:50:33,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633461,5,此用户未填写评价内容,J***3,注册会员,单机版 小D黑色,[]


In [7]:
df.columns

Index(['爬取时间(__time)', '爬取链接(__url)', '商品ID(product_id)', '评价时间(publish_time)',
       '评分（总分5分）(score)', '评价内容(content)', '评价者(author_name)',
       '评价者会员等级(author_level)', '商品sku(product_sku)', '评价标签(tags)'],
      dtype='object')

In [8]:
filterd= df['评价内容(content)'] != "此用户未填写评价内容"
data_df = df[filterd][['评价内容(content)','评分（总分5分）(score)']]

In [9]:
data_df.head()

,评价内容(content),评分（总分5分）(score)
15,一般般，一分钱一分货吧,1
18,商品质量很好，很满意，配送速度快啊，而且配送员态度也非常好。,4
19,。。。,5
22,刘慧敏提莫摸摸摸休息泽TCL退咯的一组婆婆破鼓规土局,5
25,还好还好还好还好红红火火好很好好,5


In [10]:
data = data_df.values
data

array([['一般般，一分钱一分货吧', 1],
       ['商品质量很好，很满意，配送速度快啊，而且配送员态度也非常好。', 4],
       ['。。。', 5],
       ...,
       ['没有色差，穿上很舒服，到货快。', 5],
       ['可以', 5],
       ['物有所值 客服贴心东西收到以后马上查看，发现与图片描述一致，超级喜欢，卖家发货 速度很快 ，，服务也很到位，给老板点个赞，下次还会来购买.........',
        5]], dtype=object)

In [11]:
train ,test = train_test_split(data)

In [12]:
print(train.shape)
print(test.shape)

(33316, 2)
(11106, 2)


In [13]:
tokenzier = AutoTokenizer.from_pretrained('hfl/chinese-bert-wwm') 
tokenzier

tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='hfl/chinese-bert-wwm', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [14]:
# 自定义DataLoader创建方法

def wrap_data(batch_data):
    comments, lables = [],[]
    for bdate in batch_data:
        comments.append(bdate[0])
        lables.append(int(bdate[1])-1)  # 标签取值[0-4]
    
    # 转换模型输入数据
    input_data = tokenzier(comments, return_tensors='pt', padding=True, truncation=True, max_length=512)
    labels_data = torch.tensor(lables)
    
    return input_data, labels_data

train_dl = DataLoader(train, batch_size=20, shuffle=True, collate_fn = wrap_data)
test_dl = DataLoader(test, batch_size=20, shuffle=False, collate_fn = wrap_data)

In [15]:
 # for item in test_dl:
 #     print(item)
 #     break

In [16]:
model_1 = AutoModelForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm', num_labels=5)
model_2 = AutoModelForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm', num_labels=5)

pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-bert-wwm and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-bert-wwm and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model_1 = model_1.to(device)
model_2 = model_2.to(device)
model_2.bert.trainable = False

In [18]:
# loss、optim
loss_fn1 = nn.CrossEntropyLoss()
optim1 = Adam(model_1.parameters(), lr=1e-4)

loss_fn2 = nn.CrossEntropyLoss()
optim2 = Adam(model_2.parameters(), lr=1e-4)

In [19]:
model1_train_loss_cnt = 0

for epoch in range(3):
    pbar = tqdm(train_dl)
    for input_data, labels_data in pbar:
        datas = { k:v.to(device) for k,v in input_data.items() }
        labels = labels_data.to(device)
        
        result = model_1(**datas)
        loss = loss_fn1(result.logits, labels)
        
        pbar.set_description(f'epoch:{epoch} train_loss:{loss.item():.4f}')

        writer.add_scalar("Fine Tuning Train Loss", loss, model1_train_loss_cnt)
        model1_train_loss_cnt += 1
        
        loss.backward()
        optim1.step()
        
        model_1.zero_grad()


torch.save(model_1.state_dict(),'model_1.pt')

epoch:2 train_loss:0.6043: 100%|██████████| 1666/1666 [06:26<00:00,  4.31it/s]


In [20]:
model2_train_loss_cnt = 0

for epoch in range(3):
    pbar = tqdm(train_dl)
    for input_data, labels_data in pbar:
        datas = { k:v.to(device) for k,v in input_data.items() }
        labels = labels_data.to(device)
        
        result = model_2(**datas)
        loss = loss_fn2(result.logits, labels)
        
        pbar.set_description(f'epoch:{epoch} train_loss:{loss.item():.4f}')

        writer.add_scalar("Transfer Learning Train Loss", loss, model2_train_loss_cnt)
        model2_train_loss_cnt += 1
        
        loss.backward()
        optim2.step()
        
        model_2.zero_grad()


torch.save(model_2.state_dict(),'model_2.pt')

epoch:2 train_loss:1.1347: 100%|██████████| 1666/1666 [06:27<00:00,  4.30it/s]


In [28]:
model_1.eval()
model_2.eval()
pbar = tqdm(test_dl)


correct1, correct2 = 0,0

for input_data, labels_data in pbar:
    datas = { k:v.to(device) for k,v in input_data.items() }
    labels = labels_data.to(device)

    with torch.no_grad():
        result1 = model_1(**datas)
        result2 = model_2(**datas)

    predict1 = torch.argmax(result1.logits, dim=-1)
    predict2 = torch.argmax(result2.logits, dim=-1)

    correct1 += (predict1 == labels).sum()
    correct2 += (predict2 == labels).sum()

100%|██████████| 556/556 [01:14<00:00,  7.48it/s]


In [37]:
print(f'正确率：{(correct1 / len(test_dl.dataset) * 100):.2f}/100')

正确率：94.38/100


In [46]:
# 1. 加载模型
model_1.load_state_dict(torch.load('/kaggle/working/model_1.pt'))
model_1.eval()

# 2. 准备测试数据
test_texts = ["这个产品很差", "非常差的服务"]

# 3. 使用tokenizer正确处理输入
inputs = tokenzier( 
    test_texts,
    return_tensors='pt',
    padding=True,
    truncation=True,
    max_length=512,  # 添加最大长度
    return_attention_mask=True  # 确保返回attention_mask
)

# 4. 移动数据到设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_1 = model_1.to(device)
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)  # 添加attention_mask

# 5. 运行预测
with torch.no_grad():
    outputs = model_1(
        input_ids=input_ids,
        attention_mask=attention_mask  # 传入attention_mask
    )
    # 处理SequenceClassifierOutput格式的输出
    predictions = torch.argmax(outputs.logits, dim=1)  # 从logits中获取预测结果

# 6. 打印结果
print("预测结果:", predictions.cpu().numpy())


预测结果: [4 4]
